# Tutorial for human gastrula dataset

## Entropy of Mixing functions


In [77]:
import numpy as np
from scipy.stats import fisher_exact, entropy
import multiprocessing
from functools import partial

def get_background_full(norm_adata, threshold, n_cells, n_cells_high):

    thr_per_gene = np.sum(norm_adata.X > threshold, axis=0)
    genes_filter = np.logical_and(thr_per_gene >= n_cells, thr_per_gene <= n_cells_high)
    print("Background genes: " + str(np.sum(genes_filter)))

    return genes_filter.tolist()

def perform_sub_entropy(nn_gene_expression, binary_expression, p_value, odds_ratio=2):

    entropy_nn, p_value_nn = 1, 1 
    
    if np.sum(nn_gene_expression) > 0:
        input_fisher = np.array([[np.sum(nn_gene_expression), np.sum(binary_expression)-np.sum(nn_gene_expression)],
                                [np.sum(~nn_gene_expression), np.sum(~binary_expression)-np.sum(~nn_gene_expression)]])
        oddsr_test, p_test = fisher_exact(input_fisher, alternative = 'greater')
        if p_test < p_value and oddsr_test > odds_ratio:
            p_0 = np.sum(nn_gene_expression) / len(nn_gene_expression)
            entropy_nn = entropy([p_0, 1 - p_0], base=2)
            p_value_nn = p_test

    return entropy_nn, p_value_nn

def entropy_mixing_gene(gene_expression, knn_matrix, p_value, odds_ratio, local_region, approximation):

    binary_expression = gene_expression > np.median(gene_expression)
    
    if approximation:
        knn_matrix = knn_matrix[binary_expression,:]
    
    entropies_nn = np.array([])
    p_values_nn = np.array([])
    for cell in range(knn_matrix.shape[0]):
        nn_cell = knn_matrix[cell, :]
        nn_gene_expression = binary_expression[nn_cell==1]
        entropy_sub, p_value_sub = perform_sub_entropy(nn_gene_expression, binary_expression , p_value, odds_ratio)
        entropies_nn = np.append(entropies_nn, entropy_sub)
        p_values_nn = np.append(p_values_nn, p_value_sub)

    if np.sum(entropies_nn < 1) >= local_region:
        entropy_gene = np.min(entropies_nn)
        p_value_gene = np.min(p_values_nn)
    else:
        entropy_gene = 1
        p_value_gene = 1

    return entropy_gene, p_value_gene

def entropy_mixing(norm_adata, knn_matrix, n_cores, p_value, odds_ratio, local_region, approximation):
    
    assert((knn_matrix.index.values == knn_matrix.columns.values).all)
    assert((norm_adata.obs_names == knn_matrix.index.values).all)
    #knn_matrix is pandas dataframe in same orientation as Anndata.X (cellsxgenes)
    #knn_matrix and Anndata.X should contain the cells in same order
    
    background = norm_adata.X[:, norm_adata.var["EOM_background"]]
    gene_expressions = [background[:,i].flatten() for i in range(np.shape(background)[1])]
    knn_matrix = knn_matrix.to_numpy()
    
    pool = multiprocessing.Pool(n_cores)
    temp = partial(entropy_mixing_gene, knn_matrix=knn_matrix, p_value=p_value, odds_ratio=odds_ratio, local_region=local_region, approximation=approximation)
    results = pool.map(func=temp, iterable=gene_expressions, chunksize=50)
    pool.close()
    pool.join()
    
    entropies_output = [np.NAN for i in range(len(norm_adata.var_names))]
    p_values_output = [np.NAN for i in range(len(norm_adata.var_names))]
    for index, gene_pos in enumerate(np.where(norm_adata.var["EOM_background"])[0]):
        entropies_output[gene_pos] = results[index][0]
        p_values_output[gene_pos] = results[index][1]
    
    print('\n---- Finished sucessfully! ----')

    return entropies_output, p_values_output


## Import human gastrula dataset and KNN matrix

Note that for Anndata object the count matrix is transposed (cells x genes) compared to the Seurat pipeline in R (genes x cells).

In [2]:
import scanpy as sc
import pandas as pd

human_gast_norm = sc.read_csv('/root/host_home/Documents/EntropyOfMixing/Data/norm_elmir_5_30_transposed.csv', delimiter=',')
human_gast_norm = human_gast_norm.transpose()
print(human_gast_norm)

knn_matrix = pd.read_csv('/root/host_home/Documents/EntropyOfMixing/Data/knn_matrix_elmir_5_30.csv', delimiter=',', index_col=0)


AnnData object with n_obs × n_vars = 1195 × 36570


## Entropy of mixing algorithm

**Step 1: Find background genes**

The background genes get calculated and added to the gene metadata in the AnnData object:

In [41]:
import time

t = time.perf_counter()

human_gast_norm.var["EOM_background"] = get_background_full(human_gast_norm, threshold=1, n_cells=10, n_cells_high=1000)

elapsed_time = time.perf_counter() - t
print("Execution time: " + str(np.round(elapsed_time, 2)) + "s")

#background_genes = norm_adata.var_names[norm_adata.var["EOM_background"]]

Background genes: 8812
Execution time: 0.07s


**Step 2: Calculate entropy of mixing of background genes**

The entropy and related p value for each background gene are added to the gene metadata in the AnnData object:

In [78]:
#human_gast_small = human_gast_norm[:,0:1000]
#human_gast_small = human_gast_small.copy()

t = time.perf_counter()

entropies, p_values = entropy_mixing(human_gast_norm, knn_matrix, n_cores=8, p_value=0.001, odds_ratio=2, approximation=True, local_region=2)

elapsed_time = time.perf_counter() - t
print("\nExecution Time: " + str(np.round(elapsed_time, 2)) + "s")

human_gast_norm.var["EOM_entropy"] = entropies
human_gast_norm.var["EOM_p_value"] = p_values


---- Finished sucessfully! ----

Execution Time: 289.84s


## Entropy of mixing results

We receive an extended AnnData object that contains the entropy of mixing results in its gene metadata:


In [79]:
human_gast_norm.var

,EOM_background,EOM_entropy,EOM_p_value
A1BG,False,NaN,NaN
A1BG.AS1,False,NaN,NaN
A1CF,False,NaN,NaN
A2M,True,0.0,1.570408e-07
A2M.AS1,False,NaN,NaN
...,...,...,...
ZXDC,False,NaN,NaN
ZYG11A,False,NaN,NaN
ZYG11B,True,1.0,1.000000e+00
ZYX,True,1.0,1.000000e+00
